In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adagrad, Adam, Adadelta, Nadam
from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Google Drive'da veri setimin  bulunduğu ana dizin
data_dir = '/content/drive/My Drive/dataset'

In [ ]:
# Veri artırma için ImageDataGenerator kullanma
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # ResNet-50 için veri ön işleme işlevini kullanma
)

In [ ]:
# Train ve validation verisi için veri yükleyici oluşturma
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'training_set'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # İkili sınıflandırma
)

validation_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'training_set'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Validation verisi
)


Found 8010 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
# One-hot encoding işlemi
onehot_encoder = OneHotEncoder(sparse=False)
y_onehot = onehot_encoder.fit_transform(train_generator.classes.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Model oluşturma

# ResNet-50 modelini yükleme
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


for layer in base_model.layers:
    layer.trainable = False

# Modelin üstüne özel sınıflandırma katmanlarını ekleme
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=base_model.input, outputs=output_layer)


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Erken durdurma (EarlyStopping) ve Model Kontrol Noktası (ModelCheckpoint) geri çağrıları
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

In [ ]:
# Modeli derleme
model.compile(loss='binary_crossentropy', optimizer=Adagrad(learning_rate=0.01), metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
# Adagrad optimizörü kullanımı

In [ ]:
# Modeli eğitme
history = model.fit(train_generator, epochs=30)

Epoch 1/30
251/251 [==============================] - 1502s 6s/step - loss: 0.1944 - accuracy: 0.9283 - precision: 0.9289 - recall: 0.9275
Epoch 2/30
251/251 [==============================] - 698s 3s/step - loss: 0.0955 - accuracy: 0.9643 - precision: 0.9640 - recall: 0.9645
Epoch 3/30
251/251 [==============================] - 705s 3s/step - loss: 0.0746 - accuracy: 0.9744 - precision: 0.9754 - recall: 0.9732
Epoch 4/30
251/251 [==============================] - 697s 3s/step - loss: 0.0716 - accuracy: 0.9727 - precision: 0.9716 - recall: 0.9737
Epoch 5/30
251/251 [==============================] - 693s 3s/step - loss: 0.0571 - accuracy: 0.9789 - precision: 0.9783 - recall: 0.9795
Epoch 6/30
251/251 [==============================] - 682s 3s/step - loss: 0.0547 - accuracy: 0.9799 - precision: 0.9793 - recall: 0.9805
Epoch 7/30
251/251 [==============================] - 682s 3s/step - loss: 0.0498 - accuracy: 0.9794 - precision: 0.9783 - recall: 0.9805
Epoch 8/30
251/251 [=============

In [ ]:
# Eğitim sonuçlarını saklama
train_accuracy = history.history['accuracy']
train_loss = history.history['loss']
validation_accuracy = history.history['accuracy']
validation_loss = history.history['loss']
validation_precision = history.history['precision']
validation_recall = history.history['recall']

In [ ]:
# Eğitim sonuçlarını görselleştirme
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Test verisi için veri yükleyici oluşturma
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test_set'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # İkili sınıflandırma
)

In [ ]:
# Modelin test verisi üzerinde performansını ölçme
test_metrics = model.evaluate(test_generator, return_dict=True)
print("Test Metrics:", test_metrics)


In [ ]:
# Precision ve Recall hesaplamaları
test_predictions = model.predict(test_generator)
test_predictions_binary = (test_predictions > 0.5).astype(int)
true_labels = test_generator.classes

precision = precision_score(true_labels, test_predictions_binary)
recall = recall_score(true_labels, test_predictions_binary)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [ ]:
# Test verisi için veri yükleyici oluşturma
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test_set'),
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # İkili sınıflandırma
)

In [ ]:
# Test verisi üzerinde tahminler yapma
predictions = model.predict(test_generator)

In [ ]:
# Test verisi üzerinde tahmin sonuçlarını görselleştirme
plt.figure(figsize=(12, 8))

for i in range(len(predictions)):
    if i >= 32:
        break

    plt.subplot(4, 8, i + 1)
    plt.imshow(test_generator[i][0][0])
    plt.axis('off')

    if predictions[i] >= 0.5:
        plt.title("Köpek")
    else:
        plt.title("Kedi")

plt.show()